<a href="https://www.kaggle.com/code/sevdanuropur/nlp-sentiment-analysis?scriptVersionId=201460114" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Sentiment Analysis for Amazon Reviews

**CONTENT**

* Aim of the Project 
* Dataset 
* IMPORT LIBRARIRES
* LOAD DATA SET
* Standardization
* Punctuation Removal
* Number Removal
* Stopwords
* Rare Words
* Lemmatization
* Visualization
* Wordcloud 
* Sentiment Analysis
* Modelling 
* CountVectorize()
* Logistig Regression with CountVectorize()
* TF-IDF
* Logistig Regression with TF-IDF
* TF-IDF- Ngram 
* Logistig Regression with Ngram 
* Random Forest 
* Result 
* Conclusion

**Aim of the Project**

Kozmos, a company that sells home textiles and casual wear on Amazon, aims to boost sales by analyzing customer reviews and improving its products based on recurring complaints. To achieve this, sentiment analysis will be performed on the reviews to categorize them, and a classification model will be built using the tagged data to further support decision-making.

**DATASET**

This dataset contains 4 features and consists of 5611 observations with a size of 489 KB. It includes customer reviews for a specific product group along with relevant details:

* Star: The number of stars given to the product.
* Helpful: The number of people who found the review helpful.
* Title: The title or short description of the review.
* Review: The full content of the review.

**IMPORT LIBRARIES**

In [ ]:
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns  

#### NLTK#### 

import nltk 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud
from nltk.sentiment import SentimentIntensityAnalyzer 
from textblob import Word, TextBlob

## Sklearn 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

from warnings import filterwarnings
filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

**Load Dataset**

In [ ]:
df=pd.read_excel("/kaggle/input/amazon-reviews/amazon.xlsx") 
df.head()

In [ ]:
def check_df(dataframe, head=5):
    """
    Provides a general overview of the dataset.
    
    Parameters:
    dataframe: The dataframe to be read and analyzed.
    head (int): The number of rows to display initially. The default value is 5.
    """
    print("The first {} rows of the DataFrame:".format(head))
    display(dataframe.head(head))
    print("---------------------------------")
    print("Shape information of the DataFrame:")
    display(dataframe.shape)
    print("---------------------------------")
    print("General information about the DataFrame:")
    dataframe.info()
    print("---------------------------------")
    print("Missing value information in the DataFrame:")
    display(dataframe.isnull().sum())
    print("---------------------------------")
    return True

check_df(df)


In [ ]:
# Let's look at the number of stars missing in Review u
df[df["Review"].isnull()]["Star"].value_counts()

In [ ]:
df[(df["Title"].isnull()) & (df["Review"].isnull())].shape

In [ ]:
## Lets drop the null values of Review features 

df= df[~df["Review"].isnull()]

In [ ]:
df.isnull().sum()

In [ ]:
df["Review"].head(10)

In [ ]:
star_counts = df["Star"].value_counts()

plt.figure(figsize=(6, 6))
plt.pie(star_counts, labels=star_counts.index, autopct='%1.1f%%', startangle=90, colors=['#ff9999','#66b3ff','#99ff99'])
plt.title("Star Column Distribution")
plt.axis('equal')  
plt.show()

**Standardization**

* We need to convert all letters to lowercase because converting words that start with uppercase letters but have the same meaning helps machines understand them better. By standardizing to lowercase format, we reduce the complexity of the text and ensure that similar words are treated as the same, improving the accuracy of text processing and analysis.

In [ ]:
df["Review"]= df["Review"].str.lower()

**Punctuation Removal**

* Removing punctuation marks is important in NLP as they can disrupt word analysis. By eliminating punctuation, we clean the text, allowing algorithms to focus on the content and improve the accuracy of tasks like sentiment analysis.

In [ ]:
df["Review"]= df["Review"].str.replace('[^\w\s]', '')

**Number Removal**

* Removing numbers from text is essential in NLP because they often do not contribute to the meaning in sentiment analysis or text classification. 

In [ ]:
df["Review"]= df["Review"].str.replace('\d',"")

**Stopwords**

* We remove stopwords, which are common words that carry little meaning, from the text. These words often include articles, prepositions, and conjunctions that do not add significant value to the analysis, allowing the model to focus on more meaningful terms. 


In [ ]:
sw = stopwords.words('english')

In [ ]:
# these are the words that have little meaning in a sentence
sw[:10]

In [ ]:
# This apply function eliminates stopwords from each row in the 'Review' column 
df['Review']= df['Review'].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))

**Rare Words**

* This step involves removing words that occur fewer than 1 times in the dataset. Rare words can introduce noise and don't contribute significantly to sentiment analysis. By eliminating them, we focus on more relevant terms, improving the dataset's quality for model training.

In [ ]:
# make a new series that counts the occurrences of each word.
new_df = pd.Series(' '.join(df['Review']).split()).value_counts()

# select words lest than 1 
rare_words = new_df[new_df <= 1] 

# elimate these words from each row 
df['Review'] = df['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in rare_words))

**Lemmatization:**

* Lemmatization is the process of reducing words to their base or root form, known as the lemma. For example, the words "glasses," "eye," and "eyes" all reduce to the root word "eye." This helps standardize words, making text analysis more effective by treating different forms of a word as the same.

In [ ]:
lemmatizer= WordNetLemmatizer() 

In [ ]:
def lemmatize(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
df["Review"]= df["Review"].apply(lambda x : lemmatize(x))

**Visualization**

In [ ]:
count_df= pd.Series(" ".join(df["Review"]).split()).value_counts()

count_df= count_df.reset_index()

count_df.columns=["words","tf"]

In [ ]:
count_df[count_df["tf"]>500]

In [ ]:
plt.figure(figsize=(12, 6)) 

sns.barplot(data=count_df[count_df["tf"] > 500], x="words", y="tf", palette="viridis") 

plt.xticks(rotation=45, ha='right', fontsize=10)  # Rotate and align the labels

# Add title to axes

plt.xlabel("Words", fontsize=12, labelpad=10)  # X-axis title and label padding

plt.ylabel("Term Frequency (tf)", fontsize=12, labelpad=10)  # Y-axis title

# Graph title

plt.title("Term Frequency for Words with tf > 500", fontsize=14, pad=15)

# Graph borders
sns.despine()  # We can remove the borders around the graph
# Show the graph

plt.tight_layout()  # Ensures all elements are displayed neatly

plt.show()



**Wordcloud** 

* A word cloud is a technique that visually represents the frequency of words in a text. Words that appear more frequently are displayed in larger fonts, while less common words are shown in smaller fonts

In [ ]:
text= " ".join(df["Review"])

In [ ]:
wordcloud_result= WordCloud().generate(text)

plt.imshow(wordcloud_result, interpolation="bilinear")

plt.axis("off")

plt.show()

In [ ]:
wordcloud = WordCloud(max_font_size=50,

                      max_words=100,

                      background_color="white").generate(text)

plt.figure()

plt.imshow(wordcloud, interpolation="bilinear")

plt.axis("off")

plt.show()

**Sentiment Analysis:** 

* Sentiment analysis is a natural language processing (NLP) technique used to determine the emotional tone behind a series of words. It involves classifying text as positive, negative, or neutral based on the sentiments expressed within. This analysis helps in understanding opinions, attitudes, and emotions towards a product, service, or topic, and is commonly used in social media monitoring, customer feedback, and market research.



In [ ]:
nltk.download("vader_lexicon") 

In [ ]:
sia= SentimentIntensityAnalyzer()

In [ ]:
sia.polarity_scores("The story is excellent")

#The line essentially tries to determine the sentiment of the text: whether it is positive or negative.

#In this context, a compound score greater than 0 typically indicates a positive sentiment.

In [ ]:
df["Review"][0:10].apply(lambda x : sia.polarity_scores(x))

In [ ]:
df["Review"][0:10].apply(lambda x : sia.polarity_scores(x)["compound"]).sort_values(ascending=False)

In [ ]:
# new column is maded using compound score 
df["compound_score"]=df["Review"].apply(lambda x : sia.polarity_scores(x)["compound"])

In [ ]:
# if compound score less than 0 this means this review is negative otherwise positive 
df["sentiment_label"]=df["compound_score"].apply(lambda x: "pos" if x > 0.0 else "neg")

In [ ]:
# lets see is there a rpw that compound score is positive but given star is less than 3 
# try to capture why this is happened. Maybe the user give the wrong star or review 
df[(df["compound_score"]> 0.6) & (df["Star"]<3)].head(10)

In [ ]:
df["sentiment_label"].value_counts()

In [ ]:
df.groupby("sentiment_label")["Star"].mean() 
# division of sentiment labels is reasonable

**Modelling**

In [ ]:
# change the representation of target using labelencoder
lb= LabelEncoder()
df["sentiment_label"]= lb.fit_transform(df["sentiment_label"])

In [ ]:
y= df["sentiment_label"]
X= df["Review"]

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=42, test_size=0.2)

**A crucial aspect of Natural Language Processing (NLP)** is transforming texts and words into numerical representations. This process, known as **word vectorization** enables algorithms to understand and process language in a mathematical format. Such transformations are vital for various applications, including sentiment analysis, text classification, and machine translation.

it is essential to transform our data into a measurable format, enabling us to perform mathematical operations and build machine learning models. To achieve this, we create word vectors. Commonly used methods for generating these vectors include:

* **Count Vectors:** Represents text data as the frequency of each word.

* **TF-IDF (Term Frequency-Inverse Document Frequency):** Normalizes word frequencies to highlight important words in relation to the entire dataset.

* **Word Embedding Methods:** Techniques like Word2Vec, GloVe, and BERT that capture semantic meaning by mapping words into continuous vector spaces.

These methods allow for a more nuanced understanding of language and improve the performance of various NLP tasks.

**CountVectorizer:**

* This method converts a collection of text documents into a matrix of token counts. Each unique word is represented as a column, and the entries in the matrix represent the frequency of each word in the documents. It's a straightforward way to convert text data into a numerical format for analysis and modeling.

In [ ]:
bow=CountVectorizer()

X_train_bow=bow.fit_transform(X_train).toarray()

X_test_bow=bow.transform(X_test).toarray()

In [ ]:
bow.get_feature_names_out()

In [ ]:
X_train_bow[10:14]

**Logistic Regression using CountVectorize**

In [ ]:
logreg = LogisticRegression()

logreg.fit(X_train_bow, y_train)

y_pred = logreg.predict(X_test_bow)

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")


# Cross-validation uygulama (5 katmanlı)

cv_scores = cross_val_score(logreg, X_train_bow, y_train, cv=5, scoring='accuracy')

print(f"Cross-validation scores: {cv_scores}")

print(f"Mean cross-validation score: {cv_scores.mean()}")

**TF-IDF (Term Frequency-Inverse Document Frequency):**

* This method transforms text data into a numerical representation by considering not just the frequency of words but also their importance across multiple documents. It calculates the term frequency (TF) of a word in a specific document and multiplies it by the inverse document frequency (IDF) to reduce the weight of common words and highlight rare ones. This helps capture the significance of terms in the context of the entire dataset, making it a popular choice for text analysis and information retrieval.

In [ ]:
tfidf=TfidfVectorizer()

X_train_tfidf=tfidf.fit_transform(X_train).toarray()

X_test_tfidf=tfidf.transform(X_test).toarray()

In [ ]:
tfidf.get_feature_names_out()

**Logistic Regression using TF-IDF**

In [ ]:
# Words with TF-IDF

log_model_idf = LogisticRegression().fit(X_train_tfidf, y_train)

pred_val_idf= log_model_idf.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, pred_val_idf)

print(f"Accuracy: {accuracy}")

cv_scores = cross_val_score(log_model_idf, X_train_tfidf, y_train, cv=5, scoring='accuracy')

print(f"Cross-validation scores: {cv_scores}")

print(f"Mean cross-validation score: {cv_scores.mean()}")

In [ ]:
print(classification_report(y_test, pred_val_idf))

In [ ]:
def plot_confusion_matrix(y, y_pred):
    acc = round(accuracy_score(y, y_pred), 2)
    cm = confusion_matrix(y, y_pred)
    sns.heatmap(cm, annot=True, fmt=".0f")
    plt.xlabel('y_pred')
    plt.ylabel('y')
    plt.title('Accuracy Score: {0}'.format(acc), size=10)
    plt.show()

plot_confusion_matrix(y_test, pred_val_idf)

**TF-IDF with Ngram**

In [ ]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 

X_train_tfidf_ngram=tf_idf_ngram_vectorizer.fit_transform(X_train).toarray()

X_test_tfidf_ngram =tf_idf_ngram_vectorizer.transform(X_test).toarray() 

**Logistic Regression using TF-IDF Ngram**

In [ ]:
# Words with TF-IDF
log_model_idf_ngram = LogisticRegression().fit(X_train_tfidf_ngram, y_train)

pred_val_idf_ngram= log_model_idf_ngram.predict(X_test_tfidf_ngram)

accuracy = accuracy_score(y_test, pred_val_idf_ngram)

print(f"Accuracy: {accuracy}")

cv_scores = cross_val_score(log_model_idf_ngram, X_train_tfidf_ngram, y_train, cv=5, scoring='accuracy')

print(f"Cross-validation scores: {cv_scores}")

print(f"Mean cross-validation score: {cv_scores.mean()}")

**lets Randomly select one review and look at the label**

In [ ]:
## Rastgele bir yorumun seçilmesi 

random_comment= pd.Series.sample(X_test)

random_comment

In [ ]:
random=bow.transform(random_comment).toarray()

In [ ]:
logreg.predict(random)

**Random Forest** 

In [ ]:
# Random Forest Modeli

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

cv_scores__rf_tfıdf = cross_val_score(rf_model, X_train_tfidf, y_train, cv=5, scoring='accuracy')

print(f" tfıdf Mean CV Accuracy: {cv_scores__rf_tfıdf.mean()}")

cv_scores_rf_bow = cross_val_score(rf_model, X_train_bow, y_train, cv=5, scoring='accuracy')

print(f" bow Mean CV Accuracy: {cv_scores_rf_bow.mean()}") 



# Cross-validation uygulama (5 katmanlı)

#cv_scores_rf_ngram = cross_val_score(rf_model, X_train_tfidf_ngram, y_train, cv=5, scoring='accuracy')
#print(f" ngram Mean CV Accuracy: {cv_scores_rf_ngram.mean()}")   

In [ ]:
import plotly.graph_objects as go

# Retrieve the mean accuracy scores for each model
mean_scores = [cv_scores__rf_tfıdf.mean(), cv_scores_rf_bow.mean()]

model_names = ['TF-IDF', 'BoW']

# Create a bar chart
fig = go.Figure([go.Bar(x=model_names, y=mean_scores, text=mean_scores, textposition='auto')])

# Customize the chart (title and axis labels)
fig.update_layout(
    title="Cross-Validation Mean Accuracy Scores for Different Vectorizations",
    xaxis_title="Vectorization Techniques",
    yaxis_title="Mean Accuracy",
    yaxis=dict(range=[0, 1]),  # Set the Y-axis range from 0 to 1
    plot_bgcolor='white',  # Set the background color of the chart to white
    paper_bgcolor='white',  # Set the overall background color to white
    template="plotly"  # Default template
)

# Show numerical values on hover
fig.update_traces(marker_color='lightgreen', marker_line_color='darkgreen', marker_line_width=1.5, opacity=0.8)

# Display the chart
fig.show()


**Result** 

- Based on the results, we can see that the logistic regression model built with CountVectorizer outperformed the models using TF-IDF and TF-IDF with N-grams. Additionally, when RandomForest was applied to the same vectorization methods, we observed a noticeable improvement in performance, indicating that the model's accuracy increased with this more complex algorithm.

**Conclusion**

- Hello again! It was really enjoyable for me to complete this work. I tried to concretize my analyzes on NLP Sentiment Analysis with this Amazon data set. If you liked the work I did on this subject, I am waiting for your comments. If you have any bugs or improvement suggestions, please share. Also, if you have suggestions or topics you would like to include in my future projects, let me know. I look forward to your contributions and thank you! 🚀📈